# Cleanup
This notebook is provided to help you clean up any resources you have created by running through the example. You should also go to the [CloudFormation console](https://console.aws.amazon.com/cloudformation/home) and delete the stack that you created.

In [65]:
from sagemaker import get_execution_role
import sagemaker
import boto3
import json
import sys

role = get_execution_role()
sm = boto3.Session().client(service_name='sagemaker')

### Delete feature groups

In [66]:
try:
    sm.delete_feature_group(FeatureGroupName='cc-agg-batch-fg') 
    print('deleted batch fg')
except:
    pass

try:
    sm.delete_feature_group(FeatureGroupName='cc-agg-fg') # use if needed to re-create
    print('deleted fg')
except:
    pass

deleted batch fg
deleted fg


In [67]:
sm.list_feature_groups()

{'FeatureGroupSummaries': [{'FeatureGroupName': 'demo-fg',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:355151823911:feature-group/demo-fg',
   'CreationTime': datetime.datetime(2020, 12, 2, 20, 25, 55, 593000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created'},
  {'FeatureGroupName': 'cc-agg-fg',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:355151823911:feature-group/cc-agg-fg',
   'CreationTime': datetime.datetime(2020, 12, 9, 21, 26, 58, 284000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Deleting'}],
 'ResponseMetadata': {'RequestId': 'e37d4420-dab3-48bf-b922-9cb043b25cdf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e37d4420-dab3-48bf-b922-9cb043b25cdf',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '386',
   'date': 'Wed, 09 Dec 2020 21:28:08 GMT'},
  'RetryAttempts': 0}}

### Stop the KDA SQL App

In [68]:
import boto3
kda_client = boto3.client('kinesisanalytics')

try:
    kda_client.stop_application(ApplicationName='cc-agg-app')
except:
    pass

print('Stopped the KDA SQL app')

Stopped the KDA SQL app


### Delete the KDA SQL App

In [69]:
import time
try:
    ready = False
    while not ready:
        app_desc = kda_client.describe_application(ApplicationName='cc-agg-app')['ApplicationDetail']
        if app_desc['ApplicationStatus'] == 'READY':
            ready = True
        else:
            time.sleep(15)
    create_timestamp = app_desc['CreateTimestamp']
    response = kda_client.delete_application(ApplicationName='cc-agg-app',
                                  CreateTimestamp=create_timestamp)
    print('Deleted KDA SQL app')
except:
    print('FAILED to delete KDA sql app')

Deleted KDA SQL app


### Remove the trigger from Lambda

In [70]:
import boto3

%store -r 

lambda_client = boto3.client('lambda')
cf_client = boto3.client('cloudformation')

try:
    outputs = cf_client.describe_stacks(StackName=STACK_NAME)['Stacks'][0]['Outputs']
    for o in outputs:
        if o['OutputKey'] == 'PredictLambdaFunctionARN':
            lambda_to_model_arn = o['OutputValue']
except:
    msg = f'CloudFormation stack {STACK_NAME} was not found. Please set the STACK_NAME properly and re-run this cell'
    sys.exit(ValueError(msg))
    
paginator = lambda_client.get_paginator('list_event_source_mappings')
mapping_iterator = paginator.paginate(FunctionName=lambda_to_model_arn)

for m in mapping_iterator:
    if len(m['EventSourceMappings']) > 0:
        uuid = m['EventSourceMappings'][0]['UUID']
        print(f'Deleting mapping: {uuid}...')
        lambda_client.delete_event_source_mapping(UUID=uuid)

Deleting mapping: c8d363e6-4435-48a7-865e-98360a6d62e5...


### Delete the Kinesis data stream

In [71]:
kinesis_client = boto3.client('kinesis')
try:
    kinesis_client.delete_stream(StreamName='cc-stream')
except:
    pass
print('deleted Kinesis stream')

deleted Kinesis stream


### Delete the SageMaker endpoint

In [72]:
%store -r
try:
    sm.delete_endpoint(EndpointName=endpoint_name)
except:
    pass